<a href="https://colab.research.google.com/github/rahiakela/machine-learning-research-and-practice/blob/main/hands-on-machine-learning-with-scikit-learn-keras-and-tensorflow/12-custom-models-and-training-with-tensorflow/05_tensorflow_functions_and_graphs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TensorFlow Functions and Graphs

In fact, 95% of the use cases you will encounter will not require anything other than `tf.keras` and `tf.data`.

But now it’s time to dive deeper into TensorFlow
and take a look at its lower-level Python API. This will be useful when you need extra
control to write custom loss functions, custom metrics, layers, models, initializers,
regularizers, weight constraints, and more. 

You may even need to fully control the
training loop itself, for example to apply special transformations or constraints to the
gradients (beyond just clipping them) or to use multiple optimizers for different parts
of the network.

TensorFlow’s API revolves around tensors, which flow from operation to operation—hence the name TensorFlow.

A tensor is very similar to a NumPy ndarray: it is usually
a multidimensional array, but it can also hold a scalar (a simple value, such as 42).
These tensors will be important when we create custom cost functions, custom metrics,
custom layers, and more, so let’s see how to create and manipulate them.



##Setup

In [1]:
import sys
import sklearn
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras

from tqdm.notebook import trange
from collections import OrderedDict

import numpy as np
import os
import time

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

## Loading Dataset

Let's start by loading and preparing the California housing dataset. 

In [2]:
housing = fetch_california_housing()

x_train_full, x_test, y_train_full, y_test = train_test_split(housing.data, housing.target.reshape(-1, 1), random_state=42)
x_train, x_valid, y_train, y_valid = train_test_split(x_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

##TensorFlow Function

Let’s start with a trivial function that computes the cube of its input.

In [3]:
def cube(x):
  return x ** 3

In [4]:
cube(2)

8

In [5]:
cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

Now, let’s use `tf.function()` to convert this Python function to a TensorFlow
Function:

In [6]:
tf_cube = tf.function(cube)
tf_cube

In [7]:
tf_cube(2)

<tf.Tensor: shape=(), dtype=int32, numpy=8>

In [8]:
tf_cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

Under the hood, `tf.function()` analyzed the computations performed by the `cube()` function and generated an equivalent computation graph!

Alternatively, we could have used
`tf.function` as a decorator; this is actually more common.

In [9]:
@tf.function
def my_cube(x):
  return x ** 3

The original Python function is still available via the TF Function’s `python_function` attribute, in case you ever need it:

In [10]:
my_cube.python_function(2)

8

TensorFlow optimizes the computation graph, pruning unused nodes, simplifying
expressions (e.g., 1 + 2 would get replaced with 3), and more. Once the optimized
graph is ready, the TF Function efficiently executes the operations in the graph, in the
appropriate order (and in parallel when it can).

As a result, a TF Function will usually
run much faster than the original Python function, especially if it performs complex
computations.

**Most of the time you will not really need to know more than that:
when you want to boost a Python function, just transform it into a TF Function.**

##Concrete Functions

TF Functions are polymorphic, meaning they support inputs of different types.

Every time you call a TF Function with a new combination of input types or shapes, it
generates a new concrete function, with its own graph specialized for this particular
combination. Such a combination of argument types and shapes is called an input signature.

In [11]:
concrete_function = my_cube.get_concrete_function(tf.constant(2.0))
concrete_function.graph

In [12]:
concrete_function(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

In [13]:
concrete_function(tf.constant(5.0))

<tf.Tensor: shape=(), dtype=float32, numpy=125.0>

##Function Definitions and Graphs

You can access a concrete function’s computation graph using the graph attribute.

In [14]:
ops = concrete_function.graph.get_operations()
ops

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'pow/y' type=Const>,
 <tf.Operation 'pow' type=Pow>,
 <tf.Operation 'Identity' type=Identity>]

Let’s get the list of inputs and outputs of the power operation.

In [16]:
pow_ops = ops[2]
list(pow_ops.inputs)

[<tf.Tensor 'x:0' shape=() dtype=float32>,
 <tf.Tensor 'pow/y:0' shape=() dtype=float32>]

In [17]:
pow_ops.outputs

[<tf.Tensor 'pow:0' shape=() dtype=float32>]

Note that each operation has a name.

In [18]:
concrete_function.graph.get_operation_by_name("x")

<tf.Operation 'x' type=Placeholder>

In [20]:
concrete_function.graph.get_tensor_by_name("Identity:0")

<tf.Tensor 'Identity:0' shape=() dtype=float32>

The concrete function also contains the function definition which includes the function’s signature.

In [21]:
concrete_function.function_def.signature

name: "__inference_my_cube_25"
input_arg {
  name: "x"
  type: DT_FLOAT
}
output_arg {
  name: "identity"
  type: DT_FLOAT
}

##Function Tracing

In [22]:
@tf.function
def tf_cube(x):
  print("x: ", x)
  return x ** 3

In [23]:
result = tf_cube(tf.constant(2.0))

x:  Tensor("x:0", shape=(), dtype=float32)


In [24]:
result

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

It has a
shape and a data type, but no value. Plus it has a name ("x:0"). This is because the
`print()` function is not a TensorFlow operation, so it will only run when the Python
function is traced, which happens in graph mode, with arguments replaced with symbolic
tensors (same type and shape, but no value). 

Since the `print()` function was not
captured into the graph, the next times we call `tf_cube()` with float32 scalar tensors,
nothing is printed:

In [25]:
result = tf_cube(tf.constant(3.0))

In [26]:
result = tf_cube(tf.constant(5.0))

But if we call `tf_cube()` with a tensor of a different type or shape, or with a new
Python value, the function will be traced again.

In [27]:
result = tf_cube(2)  # new Python value: trace!

x:  2


In [28]:
result = tf_cube(4)   # new Python value: trace!

x:  4


In [29]:
result = tf_cube(tf.constant([[1.0, 2.0]]))   # New shape: trace!

x:  Tensor("x:0", shape=(1, 2), dtype=float32)


In [31]:
result = tf_cube(tf.constant([[3.0,4.0], [5.0, 6.0]]))   # New shape: trace!

In [32]:
result = tf_cube(tf.constant([[7.0,8.0], [9.0, 10.0]]))   # Same shape: no trace!

In some cases, you may want to restrict a TF Function to a specific input signature.

For example, suppose you know that you will only ever call a TF Function with
batches of 28 × 28–pixel images, but the batches will have very different sizes. You
may not want TensorFlow to generate a different concrete function for each batch
size, or count on it to figure out on its own when to use None. 

In this case, you can
specify the input signature like this:

In [33]:
@tf.function(input_signature=[tf.TensorSpec([None, 28, 28], tf.float32)])
def shrink(images):
  return images[:, ::2, ::2]  # drop half the rows and columns

This TF Function will accept any float32 tensor of shape `[*, 28, 28]`, and it will reuse
the same concrete function every time:

In [34]:
img_batch_1 = tf.random.uniform(shape=[100, 28, 28])
img_batch_2 = tf.random.uniform(shape=[50, 28, 28])

preprocessed_images = shrink(img_batch_1)   # Works fine. Traces the function.
preprocessed_images = shrink(img_batch_2)   # Reuses the same concrete function

However, if you try to call this TF Function with a Python value, or a tensor of an
unexpected data type or shape, you will get an exception:

In [36]:
img_batch_3 = tf.random.uniform(shape=[2, 2, 2])
try:
  preprocessed_images = shrink(img_batch_3)  # rejects unexpected types or shapes
except ValueError as ve:
  print(ve)

Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor(
[[[0.7402308  0.33938193]
  [0.5692506  0.44811392]]

 [[0.29285502 0.4260056 ]
  [0.62890387 0.691061  ]]], shape=(2, 2, 2), dtype=float32))
  input_signature: (
    TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name=None)).


##AutoGraph Control Flow